We use are currently using Ollama, installed using the setup provided on their official website.
Finally, to run a model:
```sh
ollama run gemma3:1b

```
To self-host a small LLM Model with a size of 815MB.

To try a more powerful model:
```sh
ollama run deepseek-r1:7b
```

In [6]:
def get_review_prompt(review: str = ""):
    return ("You are an AI Model that specializes in assessing mobile application reviews.\n"
            + "You must identify only the negative experiences and return them.\n"
            + "You must only return issues that are due to the application software."
            + 'You must respond with exactly a string array containing 0 or many of the items found here: ["Functional Suitability", "Performance Efficiency", "Compatibility", "Interaction Capability", "Reliability", "Security", "Maintainability", "Flexibility", "Safety"]'
            + 'Functional Suitability: This characteristic represents the degree to which a product or system provides functions that meet stated and implied needs when used under specified conditions.'
            + 'Performance Efficiency: This characteristic represents the degree to which a product performs its functions within specified time and throughput parameters and is efficient in the use of resources (such as CPU, memory, storage, network devices, energy, materials...) under specified conditions.'
            +' Compatibility: Degree to which a product, system or component can exchange information with other products, systems or components, and/or perform its required functions while sharing the same common environment and resources.'
            +' Interaction Capability: Degree to which a product or system can be interacted with by specified users to exchange information ia the user interface to complete specific tasks in a variety of contexts of use.'
            + 'Reliability: Degree to which a system, product or component performs specified functions under specified conditions for a specified period of time.'
            +'Security: Degree to which a product or system defends against attack patterns by malicious actos and protects information and data so that persons or other products or systems have the degree of data access appropriate to their types and levels of authorization.'
            +'Maintainability: This characteristic represents the degree of effectiveness and efficiency with which a product or system can be modified to improve it, correct it or adapt it to changes in environment, and in requirements.'
            +'Flexibility: Degree to which a product can be adapted to changes in its requirements, contexts of use or system environment.'
            + 'Safety: This characteristic represents the degree to which a product under defined conditions to avoid a state in which human life, health, property, or the environment is endangered.'
            + f'Review: "{review}"')

def extract_assessment_metrics_array(response):
    response_text = response.json()["response"]
    return response_text.split("\n")[-1]
    

In [7]:
import requests

example_review = "I did like this app 'cause it keeps my daily running record. But after it's update. It's totally rugged. It took me 30 minutes to complete 6000 steps and it takes me 50 minutes to complete same steps. Moreover, I can't track Weight Management. What have you done with it?"

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "deepseek-r1:7b",
          "prompt": get_review_prompt(example_review),
          "stream": False
          },
)

print(extract_assessment_metrics_array(response))

["Functional Suitability", "Performance Efficiency"]


You are an AI Model that specializes in assessing mobile application reviews.
You must identify only the negative experiences and return them.
You must only return issues that are due to the application software.
You must respond with exactly a string array containing 0 or many of the items found here: ["Functional Suitability", "Performance Efficiency", "Compatibility", "Interaction Capability", "Reliability", "Security", "Maintainability", "Flexibility", "Safety"]
Functional Suitability: This characteristic represents the degree to which a product or system provides functions that meet stated and implied needs when used under specified conditions.
Performance Efficiency: This characteristic represents the degree to which a product performs its functions within specified time and throughput parameters and is efficient in the use of resources (such as CPU, memory, storage, network devices, energy, materials...) under specified conditions.
Compatibility: Degree to which a product, system or component can exchange information with other products, systems or components, and/or perform its required functions while sharing the same common environment and resources.
Interaction Capability: Degree to which a product or system can be interacted with by specified users to exchange information ia the user interface to complete specific tasks in a variety of contexts of use.
Reliability: Degree to which a system, product or component performs specified functions under specified conditions for a specified period of time.
Security: Degree to which a product or system defends against attack patterns by malicious actos and protects information and data so that persons or other products or systems have the degree of data access appropriate to their types and levels of authorization.
Maintainability: This characteristic represents the degree of effectiveness and efficiency with which a product or system can be modified to improve it, correct it or adapt it to changes in environment, and in requirements.
Flexibility: Degree to which a product can be adapted to changes in its requirements, contexts of use or sys tem environment.
Safety: This characteristic represents the degree to which a product under defined conditions to avoid a state in which human life, health, property, or the environment is endangered.
Review: "Love this app, especially the option to enter other exercises when phone is not able to be carried on my body such as when doing aerobics, swimming, etc. Can you make this exact app in iPhone version. Did many searches looking for this app in the Apple store, could not find anything close to it."